# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv
import gmaps.datasets
import time



# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
cities_csv=pd.read_csv("../output_data/cities.csv")
cities_csv.head()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
gmaps_key= gmaps.configure(api_key=g_key)
locations= cities_csv[["Lat","Lng"]]
weights= cities_csv["Humidity"]


In [ ]:
fig= gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations,weights=weights))
fig
plt.savefig("../resources/humidity_gmap.png")

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
cities_conditions= cities_csv.loc[(cities_csv["Max Temp"]>=70) & (cities_csv["Max Temp"]<=80)]
cities_conditions= cities_conditions.loc[cities_conditions["Cloudiness"]==0]
cities_conditions= cities_conditions.loc[cities_conditions["Wind Speed"]<=10]
cities_clean= cities_conditions.dropna()
cities_clean

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df= cities_clean

In [ ]:
#hotel_df["Hotel Name"] = ""
hotel_df

In [ ]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
print(url)

In [ ]:
#Create a params dictionary for each hotel name
criteria = {"radius":5000,
            "types": 'lodging',
            "key": g_key}



for index,row in hotel_df.iterrows():
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    lat = row["Lat"]
    lng = row["Lng"]
    criteria["location"] = f'{lat},{lng}'
    response = requests.get(url, params = criteria)
    response_json = response.json()
    
    try: 
        hotel_df.loc[index, "Hotel Name"] = response_json["results"][0]["name"]
        print("Closest hotel is found.")
        
    except:
        print("No acceptable hotel found. Trying next one")
    
    time.sleep(1)

In [ ]:
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
gmaps.configure(api_key=g_key)
selected_hotels = (locations)

fig2= gmaps.figure()
markers= gmaps.marker_layer(selected_hotels)
fig2.add_layer(markers)

# Display figure
fig2
plt.savefig("../resources/marker_gmap.png")